In [1]:
import sys
sys.path.append('../build')   # assume an out-of-source build
import exafmm as fmm
import exafmm.laplace as laplace
import exafmm.helmholtz as helmholtz
import numpy as np

In [2]:
print(fmm.__doc__)
print(laplace.__doc__)
print(helmholtz.__doc__)

exafmm's pybind11 module
exafmm's submodule for Laplace kernel
exafmm's submodule for Helmholtz kernel


#### create sources and targets

In [3]:
nsrcs = 100000
ntrgs = 100000

In [4]:
src_coords = np.random.random((nsrcs, 3))
src_charges = np.random.random(nsrcs)
trg_coords = np.random.random((ntrgs, 3))

sources_l = laplace.init_sources(src_coords, src_charges)
targets_l = laplace.init_targets(trg_coords)

In [5]:
src_charges = np.zeros(nsrcs, dtype=np.complex_)
src_charges.real = np.random.random(nsrcs)
src_charges.imag = np.random.random(nsrcs)

sources_h = helmholtz.init_sources(src_coords, src_charges)
targets_h = helmholtz.init_targets(trg_coords)

#### creat FMM instances, set FMM parameters

In [6]:
fmm_l = laplace.LaplaceFMM()
fmm_l.p = 10
fmm_l.nsurf = 6*(fmm_l.p-1)*(fmm_l.p-1) + 2
fmm_l.ncrit = 400
fmm_l.depth = 4

In [7]:
fmm_h = helmholtz.HelmholtzFMM()
fmm_h.p = 10
fmm_h.nsurf = 6*(fmm_h.p-1)*(fmm_h.p-1) + 2
fmm_h.ncrit = 400
fmm_h.depth = 4
fmm_h.wavek = 10

#### build tree

In [8]:
tree_l = laplace.build_tree(sources_l, targets_l, fmm_l)

In [9]:
print(fmm_l.r0)  # root node radius
print(fmm_l.x0)  # root node coords

0.5000031880654411
[0.499999, 0.500003, 0.500003]


In [10]:
tree_l.leafs[5].level

4

In [11]:
tree_h = helmholtz.build_tree(sources_h, targets_h, fmm_h)

In [12]:
print(fmm_h.r0)  # root node radius
print(fmm_h.x0)  # root node coords

0.5000031880654411
[0.499999, 0.500003, 0.500003]


In [13]:
type(tree_l)

exafmm.laplace.Tree

#### build interaction lists

In [14]:
fmm.init_rel_coord()  # initialize relative coordinates matrix

In [15]:
skip_P2P = False   # flag to skip near-field evaluation
laplace.build_list(tree_l, fmm_l, skip_P2P)
helmholtz.build_list(tree_h, fmm_h, skip_P2P)

#### M2L setup

In [16]:
fmm_l.M2L_setup(tree_l.nonleafs)

In [17]:
fmm_h.M2L_setup(tree_h.nonleafs)

#### pre-compute

In [18]:
fmm_l.precompute()

In [19]:
fmm_h.precompute()

#### evaluate potential

In [20]:
laplace.evaluate(tree_l, fmm_l)

P2M                  : 1.8569900e-01
M2M                  : 8.9448000e-02
P2L                  : 9.8000000e-05
M2P                  : 8.5000000e-05
P2P                  : 5.2642000e-02
M2L                  : 6.9449500e-01
L2L                  : 8.4517000e-02
L2P                  : 1.3231400e-01


array([7284.1514654 , 6722.59532207, 7489.33222565, ..., 6397.87028968,
       7072.30317255, 6350.62653948])

In [21]:
helmholtz.evaluate(tree_h, fmm_h)

P2M                  : 9.7287800e-01
M2M                  : 3.6079800e-01
P2L                  : 5.9000000e-05
M2P                  : 1.0000000e-04
P2P                  : 4.5189400e-01
M2L                  : 1.7613190e+00
L2L                  : 3.7330700e-01
L2P                  : 9.4550300e-01


array([-1047.21547504 -80.90825474j, -1181.81518058+341.43990627j,
        -843.02957461-229.82213369j, ...,  -625.62910952-279.28874219j,
       -1015.5920623  +15.49422443j,  -943.12411577 +13.98850784j])

#### check accuracy

For now, this is only valid when skip_P2P is disabled.

In [22]:
fmm_l.verify(tree_l.leafs)

[4.047325116236916e-09, 3.376563805376623e-07]

In [23]:
fmm_h.verify(tree_h.leafs)

[7.115234849748951e-08, 1.1232268921180955e-06]

#### update charges and run FMM iteratively (will be fixed later)

"""
niters = 10
for i in range(niters):
    print('-'*10 + ' iteration {} '.format(i) + '-'*10)  # print divider between iterations
    src_charges = np.random.random(nsrcs)   # generate new random charges
    fmm.update(src_charges)      # update charges
    fmm.clear()                  # clear values
    potentials = fmm.evaluate()  # evaluate potentials
    fmm.check_accuracy()         # check accuracy
"""